In [1]:
import mne

In [2]:
raw = mne.io.read_raw_gdf(r'.\BCICIV_2a_gdf\A01T.gdf', preload=True)

Extracting EDF parameters from c:\Users\emmac\OneDrive\Documents\GitHub\EEGTetris\ml\BCICIV_2a_gdf\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...


c:\Users\emmac\anaconda3\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


In [11]:
channel_names = raw.info['ch_names']

print(channel_names)

['EEG-Fz', 'EEG-0', 'EEG-1', 'EEG-2', 'EEG-3', 'EEG-4', 'EEG-5', 'EEG-C3', 'EEG-6', 'EEG-Cz', 'EEG-7', 'EEG-C4', 'EEG-8', 'EEG-9', 'EEG-10', 'EEG-11', 'EEG-12', 'EEG-13', 'EEG-14', 'EEG-Pz', 'EEG-15', 'EEG-16', 'EOG-left', 'EOG-central', 'EOG-right']


In [14]:
channels_to_keep = ['EEG-1', 'EEG-3', 'EEG-C3', 'EEG-C4', 'EEG-14', 'EEG-Pz', 'EEG-15']     # F3, F4, C3, C4, Pz, P3, P4

raw_subset = raw.copy().pick(channels_to_keep)

In [57]:
ordered_event_id = {
    '276'  : 1,
    '277'  : 2,
    '768'  : 3,
    '769'  : 4,
    '770'  : 5,
    '771'  : 6,
    '772'  : 7, 
    '783'  : 8,
    '1023' : 9,
    '1072' : 10,
    '32766': 11
}

events, event_id = mne.events_from_annotations(raw_subset, event_id = ordered_event_id)

events[:10]     # format (sample_index, previous_event_value, event_id)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


array([[    0,     0,    11],
       [    0,     0,     1],
       [29683,     0,    11],
       [29683,     0,     2],
       [49955,     0,    11],
       [49955,     0,    10],
       [91518,     0,    11],
       [91868,     0,     3],
       [92368,     0,     7],
       [93871,     0,     3]])

In [ ]:
# order_to_class_id = {
#     '1' : 'Idling Eyes Open',
#     '2' : 'Idling Eyes Closed',
#     '3' : 'Trial Start',
#     '4' : 'Cue Left',
#     '5' : 'Cue Right',
#     '6' : 'Cue Foot',
#     '7' : 'Cue Tongue', 
#     '8' : 'Cue Unknown',
#     '9' : 'Rejected Trial',
#     '10': 'Eye Movements',
#     '11': 'Next Run'
# }

# og_event_id = {
#     '276'  : 'Idling Eyes Open',
#     '277'  : 'Idling Eyes Closed',
#     '768'  : 'Trial Start',
#     '769'  : 'Cue Left',
#     '770'  : 'Cue Right',
#     '771'  : 'Cue Foot',
#     '772'  : 'Cue Tongue', 
#     '783'  : 'Cue Unknown',
#     '1023' : 'Rejected Trial',
#     '1072' : 'Eye Movements',
#     '32766': 'Next Run'
# }

In [59]:
data = raw_subset.get_data()

In [58]:
type(data), data.shape

(numpy.ndarray, (7, 672528))